# 2.3 신경망의 톱니바퀴: 텐서 연산
- **텐서 연산**(tensor operation) : 텐서 덧셈, 텐서 곱셈 등
- `layers.Dense(512, activation='relu') : 2D 텐서를 입력으로 받고 또 다른 2D 텐서를 반환하는 함수
    - `output = relu(dot(W, input) + b)` : `W`는 2D 텐서, `b`는 벡터로 둘 모두 층의 속성
    - 입력 텐서와 텐서 `W` 사이의 점곱(dot), 점곱의 결과인 2D 텐서와 벡터 `b` 사이의 덧셈(+), `relu`(렐루) 연산 총 3개의 텐서 연산이 있음
    - `relu(x)` = `max(x, 0)`

## 2.3.1 원소별 연산
- `relu` 함수와 덧셈은 **원소별 연산**(element-wise operation) : 텐서의 각 원소에 독립적으로 적용
- 고도의 병렬 구현이 가능한 연산이라는 의미

### `relu` 연산 구현

In [4]:
import numpy as np

def naive_relu(x):
    assert len(x.shape) == 2    # x는 2D 넘파이 배열
    
    x = x.copy()    # 배열 원본을 변경시키지 않기 위한 복사
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

x = np.arange(-8, 8, 3).reshape((2, 3))
print('x :')
print(x)
print()
print('relu(x) :')
print(naive_relu(x))

x :
[[-8 -5 -2]
 [ 1  4  7]]

relu(x) :
[[0 0 0]
 [1 4 7]]


### 덧셈 연산 구현

In [6]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape    # x와 y는 2D 넘파이 배열
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            x[i, j] += y[i, j]
    return x

y = np.arange(0, 6, 1).reshape((2, 3))

print('y :')
print(y)
print()
print('x + y :')
print(naive_add(x, y))

y :
[[0 1 2]
 [3 4 5]]

x + y :
[[-8 -4  0]
 [ 4  8 12]]


- 최적화된 넘파이 내장 함수로 이런 연산들을 처리할 수 있음
- BLAS(Basic Linear Algebra Subprogram) : 고도로 병렬화되고 효율적인 저수준의 텐서 조작 루틴

In [7]:
# 원소별 덧셈
z = x + y
print(z)

[[-8 -4  0]
 [ 4  8 12]]


In [9]:
# 원소별 렐루 함수
z = np.maximum(x, 0.)
print(z)

[[0. 0. 0.]
 [1. 4. 7.]]


## 2.3.2 브로드캐스팅
- 작은 텐서가 큰 텐서의 크기에 맞추어 **브로드캐스팅**(broadcasting) 됨
    1. 큰 텐서의 `ndim`에 맞도록 작은 텐서에 (브로드캐스팅 축이라고 부르는) 축이 추가됨
    2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복
- 예제
    - `X.shape == (32, 10)`이고 `y.shape == (10, )`인 경우
        1. `y`에 비어 있는 첫 번째 축을 추가하여 크기를 `(1, 10)`으로 만듬
        2. 32번 반복하여 텐서 `Y`의 크기를 `(32, 10)`으로 만듬
            - `Y[i, :] == y for i in range(0, 32)`
- 그러나 새로운 텐서가 만들어지면 비효율적이므로 실제로는 어떤 2D 텐서도 만들어지지 않음

### 브로드캐스팅 구현

In [10]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2    # x는 2D 넘파이 배열
    assert len(y.shape) == 1    # y는 1D 넘파이 배열
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
    return x

x = np.arange(0, 6).reshape((2, 3))
y = np.arange(-5, 6, 5).reshape((-1))
print('x :', x, 'y :', y)

x : [[0 1 2]
 [3 4 5]] y : [-5  0  5]


In [11]:
print(naive_add_matrix_and_vector(x, y))

[[-5  1  7]
 [-2  4 10]]


`(a, b, ..., n, n+1, ..., m)` 크기의 텐서와 `(n, n+1, ..., m)` 크기의 텐서 사이에 브로드캐스팅으로 원소별 연산을 적용할 수 있음

In [13]:
# 브로드캐스팅으로 원소별 maximum 연산 적용
x = np.random.random((64, 3, 32, 10))
y = np.random.random((32, 10))

z = np.maximum(x, y)

## 2.3.3 텐서 점곰
- **텐서 곱셈**(tensor product)라고도 부르는 점곱 연산은 가장 잘 사용되고 유용한 텐서 연산
- 텐서플로에서는 `tf.matmul(x, y)`처럼 사용, 파이썬 3.5 이상에서는 `x @ y`로 계산
    - 원소별 곱셈은 `*` 연산자 사용

In [14]:
x = np.array([1, 2, 3])
y = np.array([-1, -2, -3])

z = np.dot(x, y)
print(z)

-14


In [15]:
z = x @ y
print(z)

-14


### 벡터끼리의 점곱 연산 구현

In [16]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1    # x, y는 넘파이 벡터
    assert x.shape[0] == y.shape[0]
    
    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z

In [17]:
z = naive_vector_dot(x, y)
print(z)

-14.0


### 행렬과 벡터 사이의 점곱 연산 구현
- 행렬 x와 y 사이에서 점곱이 일어나므로 벡터가 반환

In [18]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2    # x는 넘파이 행렬
    assert len(y.shape) == 1    # y는 넘파이 벡터
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i, j] * y[j]
    return z

x = np.arange(1, 7, 1).reshape((2, 3))
y = np.array([-1, -2, -3])

z = naive_matrix_vector_dot(x, y)
print(z)

[-14. -32.]


In [19]:
def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2    # x는 넘파이 행렬
    assert len(y.shape) == 1    # y는 넘파이 벡터
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

z = naive_matrix_vector_dot(x, y)
print(z)

[-14. -32.]


- x, y 둘 중 하나의 `ndim`이 1보다 크면 `dot(x, y)`와 `dot(y, x)`는 다르다.
- 둘 다 `ndim`이 1이면 교환 법칙이 성립한다.

### 행렬끼리의 점곱 연산 구현

In [21]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

x = np.arange(1, 7, 1).reshape((2, 3))
y = np.arange(7, 1, -1).reshape((3, 2))

z = naive_matrix_dot(x, y)
print(z)

[[26. 20.]
 [71. 56.]]


- 크기를 맞추는 동일한 규칙을 따르면 고차원 텐서 간의 점곱을 할 수 있음
    - `(a, b, c, d) · (d, ) → (a, b, c)`
    - `(a, b, c, d) · (d, e) → (a, b, c, e)`

## 2.3.4 텐서 크기 변환
- **텐서 크기 변환**(tensor reshaping)은 꼭 알아둬야 할 텐서 연산

In [22]:
x = np.array([[0, 1],
             [2, 3],
             [4, 5]])
print(x.shape)

(3, 2)


In [23]:
x = x.reshape((6, 1))
print(x)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]]


- 자주 사용하는 크기 변환은 **전치**(transpose)로 행과 열을 바꾸는 것
    - `x[i, :]`이 `x[:, i]`가 됨

In [25]:
print(y)

[[7 6]
 [5 4]
 [3 2]]


In [26]:
print(np.transpose(y))

[[7 5 3]
 [6 4 2]]


## 2.3.5 텐서 연산의 기하학적 해석
- 모든 텐서는 좌표 포인트로 해석될 수 있기 때문에 기하학적 해석이 가능
- 아핀 변환(affine transformation; 거리의 비율과 직선의 평행을 유지하는 이동, 스케일링, 회전 등), 회전, 스케일링(scaling) 등 기본적인 기하학적 연산은 텐서 연산으로 구현할 수 있음
- 예를 들어 `theta` 각도로 2D 벡터를 회전하는 것은 2x2 행렬 `R = [u, v]`를 점곱하여 구현할 수 있음
    - `u, v`는 동일 평면상의 벡터이며 `u = [cos(theta), sin(theta)]`, `v = [-sin(theta), cos(theta)]`

## 2.3.6 딥러닝의 기하학적 해석
- 복잡하고 심하게 꼬여 있는 데이터의 매니폴드에 대한 깔끔한 표현을 찾는 일을 딥러닝의 심층 네트워크이 각 층이 함